In [1]:
pip install pandas numpy opencv-python torch torchvision transformers


   ---------------------------------------- 0.0/9.5 MB ? eta -:--:--
   --- ------------------------------------ 0.8/9.5 MB 6.7 MB/s eta 0:00:02
   -------- ------------------------------- 2.1/9.5 MB 6.2 MB/s eta 0:00:02
   ------------------ --------------------- 4.5/9.5 MB 8.1 MB/s eta 0:00:01
   ------------------------- -------------- 6.0/9.5 MB 8.0 MB/s eta 0:00:01
   ------------------------------- -------- 7.3/9.5 MB 7.8 MB/s eta 0:00:01
   ------------------------------------- -- 8.9/9.5 MB 7.8 MB/s eta 0:00:01
   ---------------------------------------- 9.5/9.5 MB 7.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------- ----------- 1.6/2.2 MB 7.6 MB/s eta 0:00:01
   ---------------------------------------- 2.2/2.2 MB 7.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
train_data = pd.read_csv('dataset/train.csv') 


In [6]:
train_data['image_path'] = 'train_images/' + train_data['image_link'].apply(lambda url: url.split("/")[-1])


In [7]:
test_data = pd.read_csv('dataset/test.csv') 

In [8]:
test_data['image_path'] = 'test_images/' + test_data['image_link'].apply(lambda url: url.split("/")[-1])


In [9]:
entity_unit_map = {
    'width': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'depth': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'height': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'item_weight': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
    'maximum_weight_recommendation': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
    'voltage': {'kilovolt', 'millivolt', 'volt'},
    'wattage': {'kilowatt', 'watt'},
    'item_volume': {'centilitre', 'cubic foot', 'cubic inch', 'cup', 'decilitre', 'fluid ounce', 'gallon', 
                    'imperial gallon', 'litre', 'microlitre', 'millilitre', 'pint', 'quart'}
}

In [10]:
def get_default_unit(entity_name):
    if entity_name in entity_unit_map:
        return next(iter(entity_unit_map[entity_name]))  # Using the first unit in the set
    return ""

In [11]:
test_data['default_unit'] = test_data['entity_name'].apply(get_default_unit)

In [28]:
import torch
from torchvision import models, transforms
from PIL import Image
from tqdm import tqdm
import pandas as pd


In [29]:
# Loading the model
model = models.resnet50(weights='IMAGENET1K_V1')
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [30]:
# Defining image transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [31]:
# Extracting features from images
def extract_image_features(image_path):
    try:
        image = Image.open(image_path).convert('RGB')
        image = transform(image).unsqueeze(0)  # Adding batch dimension
        with torch.no_grad():
            features = model(image)
        return features.squeeze().numpy()  # Returning as numpy array
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return None

In [32]:
# Applying feature extraction to train data with progress tracking
def process_data(data):
    features = []
    for image_path in tqdm(data['image_path'], desc="Processing images"):
        feature = extract_image_features(image_path)
        features.append(feature)
    return features

In [33]:
# Applying feature extraction to train and test data
train_data['image_features'] = process_data(train_data)
test_data['image_features'] = process_data(test_data)

Processing images:   0%|          | 1098/263859 [01:29<5:16:13, 13.85it/s]

Error processing train_images/41NH8WgeBOL.jpg: image file is truncated (2 bytes not processed)


Processing images:  27%|██▋       | 70597/263859 [1:38:42<4:10:48, 12.84it/s]

Error processing train_images/41cxahpsHuL.jpg: image file is truncated (5 bytes not processed)


Processing images:  27%|██▋       | 70609/263859 [1:38:43<4:28:26, 12.00it/s]

...

Processing images:  98%|█████████▊| 128610/131187 [2:56:17<04:10, 10.28it/s]

Error processing test_images/81eAv-jpiOL.jpg: [Errno 2] No such file or directory: 'test_images/81eAv-jpiOL.jpg'


Processing images: 100%|█████████▉| 130757/131187 [3:00:45<00:44,  9.61it/s]

Error processing test_images/91Y5t1M1BTL.jpg: image file is truncated (2 bytes not processed)


Processing images: 100%|██████████| 131187/131187 [3:01:41<00:00, 12.03it/s]


In [34]:
#Training a model on the extracted image features
from sklearn.ensemble import GradientBoostingRegressor

In [56]:
import numpy as np
# Prepareing the training data
X_train = np.array([features for features in train_data['image_features'] if features is not None])  # Features
y_train = train_data.loc[train_data['image_features'].notnull(), 'entity_value']  # Labels

In [58]:
import re

def extract_numeric_value(value_with_unit):
    if pd.isna(value_with_unit):
        return np.nan
    match = re.match(r"([0-9.]+)\s*([a-zA-Z]+)", value_with_unit)
    if match:
        value, unit = match.groups()
        value = float(value)
        return value  # Adding unit conversion logic here if needed
    return np.nan

train_data['numeric_value'] = train_data['entity_value'].apply(extract_numeric_value)


In [59]:
def convert_to_gram(value, unit):
    conversion_factors = {
        'kilogram': 1000,
        'gram': 1,
        'milligram': 0.001,
        'ounce': 28.3495,
        'pound': 453.592,
        'ton': 1e6
    }
    return value * conversion_factors.get(unit, 1)


def convert_value(value_with_unit):
    if pd.isna(value_with_unit):
        return np.nan
    match = re.match(r"([0-9.]+)\s*([a-zA-Z]+)", value_with_unit)
    if match:
        value, unit = match.groups()
        value = float(value)
        return convert_to_gram(value, unit)  # Converting to grams
    return np.nan

train_data['numeric_value'] = train_data['entity_value'].apply(convert_value)


In [61]:
# Filtering out any rows where `numeric_value` is NaN
train_data = train_data.dropna(subset=['numeric_value', 'image_features'])

X_train = np.array([features for features in train_data['image_features']])
y_train = train_data['numeric_value'].values


In [69]:
pip install joblib


Note: you may need to restart the kernel to use updated packages.


In [70]:
import joblib
import logging
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split

In [71]:
# Configuring logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [72]:
# Ensuring X_train and y_train have the same length
if len(X_train) != len(y_train):
    logger.error("Feature and target data lengths do not match.")
    raise ValueError("Feature and target data lengths do not match.")

In [74]:
# Ensureing that X_train is aligned with y_train after dropping NaNs
# Filtering out rows with NaN targets from X_train
valid_indices = ~np.isnan(y_train)
X_train = np.array(X_train)[valid_indices]

In [75]:
# Checking the dimensions again
if len(X_train) != len(y_train):
    logger.error("Mismatch in lengths of X_train and y_train after cleaning.")
    raise ValueError("Mismatch in lengths of X_train and y_train after cleaning.")

In [76]:
# Initializing and training the Gradient Boosting Regressor
regressor = GradientBoostingRegressor()
try:
    logger.info("Training the model...")
    regressor.fit(X_train, y_train)
    logger.info("Model training completed successfully.")
except Exception as e:
    logger.error(f"An error occurred during model training: {e}")
    raise

INFO:__main__:Training the model...
INFO:__main__:Model training completed successfully.


In [124]:
# Ensureing test_data has valid image features for prediction
X_test = test_data['image_features'].dropna().tolist()  

# Predicting the values using the trained regressor only for rows with valid image features
predictions = regressor.predict(X_test)

# Creating a new column 'value_pred' and set default empty values
test_data['value_pred'] = ""

# Assigning predictions back to the original test data (only where image features are valid)
test_data.loc[test_data['image_features'].notna(), 'value_pred'] = predictions


In [131]:
def format_prediction(value, unit):
    if pd.isna(value) or value == "":
        return ""  
    formatted_value = abs(value)

    return f"{formatted_value:.2f} {unit}"

# Applying the formatting function to the test data
test_data['prediction'] = test_data.apply(lambda row: format_prediction(row['value_pred'], row['default_unit']), axis=1)


In [132]:
# Ensuring the output contains 'index' and 'prediction' columns, matching the expected format
output_df = test_data[['index', 'prediction']]

# Saving the predictions to a CSV file named 'test_out.csv'
output_df.to_csv('test_out.csv', index=False)


In [134]:
#Sanity Checking
!python src/sanity.py --test_filename dataset/test.csv --output_filename test_out.csv


Parsing successfull for file: test_out.csv
